In [42]:
import pandas as pd
import numpy as np
from sklearn.neural_network import BernoulliRBM, MLPClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.pipeline import Pipeline

# Reading file

In [43]:
df = pd.read_csv('../dataset.csv')

In [44]:
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close,Stock,Average,Return,Max,Min,Day,y
0,2006-01-03,0.710553,0.707904,0.705324,0.714383,5307000,21.399132,A,0.711972,NaN,1,5,0.166667,NaN
1,2006-01-04,0.715643,0.716495,0.723974,0.717436,4195800,21.456621,A,0.720884,0.002687,1,5,0.333333,2.0
2,2006-01-05,0.712250,0.738488,0.726687,0.747286,4835400,22.018748,A,0.733739,0.026198,1,5,0.500000,3.0
3,2006-01-06,0.747879,0.749485,0.748389,0.753392,6146300,22.133729,A,0.752507,0.005222,1,5,0.666667,2.0
4,2006-01-09,0.752969,0.749828,0.759580,0.750000,4082800,22.069850,A,0.755849,-0.002886,1,5,0.000000,2.0


In [45]:
df[['Stock','y']].groupby('Stock').describe()

y                                             
        count      mean       std  min  25%  50%  75%  max
Stock                                                     
A      1258.0  2.018283  0.962536  0.0  2.0  2.0  3.0  4.0
AAL    1258.0  1.953100  1.485607  0.0  1.0  2.0  3.0  4.0
AAN    1258.0  2.006359  0.997191  0.0  1.0  2.0  3.0  4.0
AAOI    318.0  1.937107  1.279223  0.0  1.0  2.0  3.0  4.0
AAP    1258.0  2.014308  0.956385  0.0  1.0  2.0  3.0  4.0

In [46]:
df = df[df.Stock == 'A']

In [47]:
df.describe()

,Open,High,Low,Close,Volume,Adj Close,Average,Return,Max,Min,Day,y
count,1259.000000,1259.000000,1259.000000,1259.000000,1.259000e+03,1259.000000,1259.000000,1258.000000,1259.0,1259.0,1259.000000,1258.000000
mean,0.622552,0.626594,0.624178,0.624213,4.357144e+06,20.688205,0.626183,0.000429,1.0,5.0,0.336643,2.018283
std,0.220899,0.223228,0.221613,0.221125,2.021792e+06,4.335443,0.223169,0.022797,0.0,0.0,0.233183,0.962536
min,0.000000,0.000000,0.000000,0.000000,9.693000e+05,8.433733,0.000000,-0.108450,1.0,5.0,0.000000,0.000000
25%,0.540210,0.543814,0.537979,0.538331,3.036550e+06,19.033525,0.542720,-0.009621,1.0,5.0,0.166667,2.000000
50%,0.676960,0.682818,0.678196,0.679444,3.967500e+06,21.728388,0.681292,0.001001,1.0,5.0,0.333333,2.000000
75%,0.781812,0.790378,0.782299,0.782734,5.117300e+06,23.824879,0.786014,0.011127,1.0,5.0,0.500000,3.000000
max,1.000000,1.000000,1.000000,1.000000,2.068870e+07,28.419782,1.000000,0.138790,1.0,5.0,0.666667,4.000000


In [48]:
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close,Stock,Average,Return,Max,Min,Day,y
0,2006-01-03,0.710553,0.707904,0.705324,0.714383,5307000,21.399132,A,0.711972,NaN,1,5,0.166667,NaN
1,2006-01-04,0.715643,0.716495,0.723974,0.717436,4195800,21.456621,A,0.720884,0.002687,1,5,0.333333,2.0
2,2006-01-05,0.712250,0.738488,0.726687,0.747286,4835400,22.018748,A,0.733739,0.026198,1,5,0.500000,3.0
3,2006-01-06,0.747879,0.749485,0.748389,0.753392,6146300,22.133729,A,0.752507,0.005222,1,5,0.666667,2.0
4,2006-01-09,0.752969,0.749828,0.759580,0.750000,4082800,22.069850,A,0.755849,-0.002886,1,5,0.000000,2.0


# X & Y

In [49]:
k=20 #Max lags

X = pd.DataFrame()
X.loc[:,'Day'] = df.loc[:,'Day'].iloc[k:].reset_index(drop=True)
y = df['y'].iloc[k:].reset_index(drop=True)
y2 = df['Return'].iloc[k:].reset_index(drop=True)

for i in range(k):
        X.loc[:,('Lag %i' % (i+1))] = df.loc[:,'Average'].iloc[k-1-i:-1-i].reset_index(drop=True)
        
print("shape of X is ", X.shape, " and shape of y is ", y.shape)

shape of X is  (1239, 21)  and shape of y is  (1239,)


In [50]:
X.head()

,Day,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5,Lag 6,Lag 7,Lag 8,Lag 9,...,Lag 11,Lag 12,Lag 13,Lag 14,Lag 15,Lag 16,Lag 17,Lag 18,Lag 19,Lag 20
0,0.333333,0.738452,0.744280,0.742137,0.741366,0.744880,0.745994,0.745137,0.753106,0.762876,...,0.722170,0.734510,0.752250,0.766561,0.755249,0.755849,0.752507,0.733739,0.720884,0.711972
1,0.500000,0.748136,0.738452,0.744280,0.742137,0.741366,0.744880,0.745994,0.745137,0.753106,...,0.736738,0.722170,0.734510,0.752250,0.766561,0.755249,0.755849,0.752507,0.733739,0.720884
2,0.666667,0.748565,0.748136,0.738452,0.744280,0.742137,0.741366,0.744880,0.745994,0.745137,...,0.762876,0.736738,0.722170,0.734510,0.752250,0.766561,0.755249,0.755849,0.752507,0.733739
3,0.000000,0.744537,0.748565,0.748136,0.738452,0.744280,0.742137,0.741366,0.744880,0.745994,...,0.753106,0.762876,0.736738,0.722170,0.734510,0.752250,0.766561,0.755249,0.755849,0.752507
4,0.166667,0.753364,0.744537,0.748565,0.748136,0.738452,0.744280,0.742137,0.741366,0.744880,...,0.745137,0.753106,0.762876,0.736738,0.722170,0.734510,0.752250,0.766561,0.755249,0.755849


In [51]:
y.head()

0    4.0
1    1.0
2    2.0
3    3.0
4    1.0
Name: y, dtype: float64

# X with Open, High, Low and Close

In [7]:
X = pd.DataFrame()
X = df.loc[:,['Open','High','Low','Close']].iloc[0:-k].reset_index(drop=True)

# X with 0 and 1 (randomly)

In [134]:
X= pd.DataFrame(np.random.binomial(1,0.5,size=(y.shape[0],4)))

# Spliting

In [52]:
split_var = int(X.shape[0]/4*3)
X_train, X_test, y_train, y_test = [X.iloc[:split_var,:],X.iloc[split_var:,:],y.iloc[:split_var],y.iloc[split_var:]]
print("shape of X_train is ", X_train.shape, " and shape of y_train is ", y_train.shape)
print("shape of X_test is ", X_test.shape, " and shape of y_test is ", y_test.shape)


#Exporting files to use in R later
X_train.to_csv('../csv/X_train_' + df.iloc[1].loc['Stock'] + '.csv')
X_test.to_csv('../csv/X_test_' + df.iloc[1].loc['Stock'] + '.csv')
y_train.to_csv('../csv/y_train_' + df.iloc[1].loc['Stock'] + '.csv')
y_test.to_csv('../csv/y_test_' + df.iloc[1].loc['Stock'] + '.csv')

shape of X_train is  (929, 21)  and shape of y_train is  (929,)
shape of X_test is  (310, 21)  and shape of y_test is  (310,)


In [40]:
X_train.head()

,Day,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5,Lag 6,Lag 7,Lag 8,Lag 9,...,Lag 11,Lag 12,Lag 13,Lag 14,Lag 15,Lag 16,Lag 17,Lag 18,Lag 19,Lag 20
0,0.333333,0.455884,0.448113,0.454486,0.474879,0.482526,0.473193,0.456624,0.457816,0.471672,...,0.492024,0.537497,0.531412,0.567388,0.598100,0.607434,0.614711,0.618617,0.609119,0.586835
1,0.500000,0.449799,0.455884,0.448113,0.454486,0.474879,0.482526,0.473193,0.456624,0.457816,...,0.464970,0.492024,0.537497,0.531412,0.567388,0.598100,0.607434,0.614711,0.618617,0.609119
2,0.666667,0.458597,0.449799,0.455884,0.448113,0.454486,0.474879,0.482526,0.473193,0.456624,...,0.471672,0.464970,0.492024,0.537497,0.531412,0.567388,0.598100,0.607434,0.614711,0.618617
3,0.000000,0.459337,0.458597,0.449799,0.455884,0.448113,0.454486,0.474879,0.482526,0.473193,...,0.457816,0.471672,0.464970,0.492024,0.537497,0.531412,0.567388,0.598100,0.607434,0.614711
4,0.166667,0.451361,0.459337,0.458597,0.449799,0.455884,0.448113,0.454486,0.474879,0.482526,...,0.456624,0.457816,0.471672,0.464970,0.492024,0.537497,0.531412,0.567388,0.598100,0.607434


In [41]:
y_train.head()

0    2.0
1    3.0
2    1.0
3    2.0
4    2.0
Name: y, dtype: float64

# Table

lags | C |n_iter| batch_size | hidden_units | learning_rate | Scr_training | Scr_test
--- | --- | ---
1|1|10|90|100 | .1 |0.479321314952|0.51746031746
1|1|10|90|100| .01|0.479321314952|0.51746031746
2|1|10|90|100| .01|0.478768577495|0.51746031746
3|1|10|90|100|.01|0.478768577495|0.519108280255
4|1|10|90|100|.01|0.47821466525|0.519108280255
5|1|10|90|100|.01|0.477659574468|0.519108280255
5|1|10|90|100|.001|0.477659574468|0.519108280255
5|100|10|90|100|.001|0.479321314952|0.51746031746
5|1000|10|90|100|.001|0.479321314952|0.51746031746

# Running RBM-logistic

In [10]:
# for rbm
learning_rate = 0.001
hidden_units = 1000
batch_size = 90
n_iter = 1000

# for logistic
C = 100

In [11]:
logistic = LogisticRegression(C=C)
mlp = MLPClassifier()
rbm0 = BernoulliRBM(random_state=0, verbose=True)
rbm1 = BernoulliRBM(random_state=0, verbose=True)

classifier0 = Pipeline([('rbm', rbm0),('mlp',mlp)])
classifier1 = Pipeline([('rbm', rbm1),('logistic',logistic)])

rbm0.learning_rate = learning_rate
rbm0.n_components=hidden_units
rbm0.batch_size = batch_size
rbm0.n_iter = n_iter
rbm1.learning_rate = learning_rate
rbm1.n_components=hidden_units
rbm1.batch_size = batch_size
rbm1.n_iter = n_iter

mlp.hidden_layer_sizes = (100,)

classifier0.fit(X=X_train, y=y_train);
classifier1.fit(X=X_train, y=y_train);

print("Score for RBM-MLP")
print("score on training: ",classifier0.score(X_train,y_train))
print("score on test: ",classifier0.score(X_test,y_test))
print("\nScore for RBM-logistic")
print("score on training: ",classifier1.score(X_train,y_train))
print("score on test: ",classifier1.score(X_test,y_test))
print("\nDifferences in the score between both models")
print("For training: ", classifier0.score(X_train,y_train)-classifier1.score(X_train,y_train))
print("For Testing: ", classifier0.score(X_test,y_test)-classifier1.score(X_test,y_test))

[BernoulliRBM] Iteration 1, pseudo-likelihood = -22.61, time = 0.10s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -22.92, time = 0.24s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -23.11, time = 0.23s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -23.19, time = 0.22s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -23.16, time = 0.22s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -23.13, time = 0.21s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -23.19, time = 0.21s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -23.21, time = 0.22s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -23.16, time = 0.21s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -23.15, time = 0.21s
[BernoulliRBM] Iteration 11, pseudo-likelihood = -23.21, time = 0.22s
[BernoulliRBM] Iteration 12, pseudo-likelihood = -23.13, time = 0.22s
[BernoulliRBM] Iteration 13, pseudo-likelihood = -23.05, time = 0.21s
[BernoulliRBM] Iteration 14, pseudo-likelihood = -23.12, time = 0.21s
[BernoulliRBM] Iteration 15, 

[BernoulliRBM] Iteration 118, pseudo-likelihood = -22.32, time = 0.21s
[BernoulliRBM] Iteration 119, pseudo-likelihood = -22.25, time = 0.22s
[BernoulliRBM] Iteration 120, pseudo-likelihood = -22.17, time = 0.22s
[BernoulliRBM] Iteration 121, pseudo-likelihood = -22.13, time = 0.23s
[BernoulliRBM] Iteration 122, pseudo-likelihood = -22.15, time = 0.21s
[BernoulliRBM] Iteration 123, pseudo-likelihood = -22.18, time = 0.22s
[BernoulliRBM] Iteration 124, pseudo-likelihood = -22.16, time = 0.21s
[BernoulliRBM] Iteration 125, pseudo-likelihood = -22.12, time = 0.21s
[BernoulliRBM] Iteration 126, pseudo-likelihood = -22.13, time = 0.23s
[BernoulliRBM] Iteration 127, pseudo-likelihood = -22.12, time = 0.21s
[BernoulliRBM] Iteration 128, pseudo-likelihood = -22.10, time = 0.21s
[BernoulliRBM] Iteration 129, pseudo-likelihood = -22.09, time = 0.23s
[BernoulliRBM] Iteration 130, pseudo-likelihood = -22.08, time = 0.23s
[BernoulliRBM] Iteration 131, pseudo-likelihood = -22.09, time = 0.23s
[Berno

[BernoulliRBM] Iteration 234, pseudo-likelihood = -17.52, time = 0.23s
[BernoulliRBM] Iteration 235, pseudo-likelihood = -17.42, time = 0.22s
[BernoulliRBM] Iteration 236, pseudo-likelihood = -17.32, time = 0.22s
[BernoulliRBM] Iteration 237, pseudo-likelihood = -17.27, time = 0.23s
[BernoulliRBM] Iteration 238, pseudo-likelihood = -17.24, time = 0.23s
[BernoulliRBM] Iteration 239, pseudo-likelihood = -17.19, time = 0.23s
[BernoulliRBM] Iteration 240, pseudo-likelihood = -17.16, time = 0.23s
[BernoulliRBM] Iteration 241, pseudo-likelihood = -17.14, time = 0.23s
[BernoulliRBM] Iteration 242, pseudo-likelihood = -16.94, time = 0.22s
[BernoulliRBM] Iteration 243, pseudo-likelihood = -16.85, time = 0.22s
[BernoulliRBM] Iteration 244, pseudo-likelihood = -16.96, time = 0.22s
[BernoulliRBM] Iteration 245, pseudo-likelihood = -16.85, time = 0.21s
[BernoulliRBM] Iteration 246, pseudo-likelihood = -16.74, time = 0.23s
[BernoulliRBM] Iteration 247, pseudo-likelihood = -16.62, time = 0.22s
[Berno

[BernoulliRBM] Iteration 350, pseudo-likelihood = -14.05, time = 0.24s
[BernoulliRBM] Iteration 351, pseudo-likelihood = -14.16, time = 0.23s
[BernoulliRBM] Iteration 352, pseudo-likelihood = -13.95, time = 0.22s
[BernoulliRBM] Iteration 353, pseudo-likelihood = -13.88, time = 0.22s
[BernoulliRBM] Iteration 354, pseudo-likelihood = -13.88, time = 0.23s
[BernoulliRBM] Iteration 355, pseudo-likelihood = -13.90, time = 0.23s
[BernoulliRBM] Iteration 356, pseudo-likelihood = -13.96, time = 0.22s
[BernoulliRBM] Iteration 357, pseudo-likelihood = -14.04, time = 0.21s
[BernoulliRBM] Iteration 358, pseudo-likelihood = -13.95, time = 0.22s
[BernoulliRBM] Iteration 359, pseudo-likelihood = -13.86, time = 0.21s
[BernoulliRBM] Iteration 360, pseudo-likelihood = -13.99, time = 0.21s
[BernoulliRBM] Iteration 361, pseudo-likelihood = -13.92, time = 0.23s
[BernoulliRBM] Iteration 362, pseudo-likelihood = -13.95, time = 0.21s
[BernoulliRBM] Iteration 363, pseudo-likelihood = -14.00, time = 0.22s
[Berno

[BernoulliRBM] Iteration 466, pseudo-likelihood = -13.88, time = 0.21s
[BernoulliRBM] Iteration 467, pseudo-likelihood = -13.89, time = 0.21s
[BernoulliRBM] Iteration 468, pseudo-likelihood = -13.68, time = 0.22s
[BernoulliRBM] Iteration 469, pseudo-likelihood = -13.71, time = 0.23s
[BernoulliRBM] Iteration 470, pseudo-likelihood = -13.94, time = 0.23s
[BernoulliRBM] Iteration 471, pseudo-likelihood = -13.91, time = 0.21s
[BernoulliRBM] Iteration 472, pseudo-likelihood = -13.78, time = 0.21s
[BernoulliRBM] Iteration 473, pseudo-likelihood = -13.87, time = 0.22s
[BernoulliRBM] Iteration 474, pseudo-likelihood = -13.99, time = 0.21s
[BernoulliRBM] Iteration 475, pseudo-likelihood = -13.99, time = 0.21s
[BernoulliRBM] Iteration 476, pseudo-likelihood = -13.91, time = 0.22s
[BernoulliRBM] Iteration 477, pseudo-likelihood = -13.79, time = 0.22s
[BernoulliRBM] Iteration 478, pseudo-likelihood = -13.67, time = 0.22s
[BernoulliRBM] Iteration 479, pseudo-likelihood = -13.74, time = 0.22s
[Berno

...

[BernoulliRBM] Iteration 512, pseudo-likelihood = -13.72, time = 0.22s
[BernoulliRBM] Iteration 513, pseudo-likelihood = -13.88, time = 0.24s
[BernoulliRBM] Iteration 514, pseudo-likelihood = -13.94, time = 0.25s
[BernoulliRBM] Iteration 515, pseudo-likelihood = -13.83, time = 0.23s
[BernoulliRBM] Iteration 516, pseudo-likelihood = -13.60, time = 0.25s
[BernoulliRBM] Iteration 517, pseudo-likelihood = -13.67, time = 0.21s
[BernoulliRBM] Iteration 518, pseudo-likelihood = -13.83, time = 0.22s
[BernoulliRBM] Iteration 519, pseudo-likelihood = -13.93, time = 0.22s
[BernoulliRBM] Iteration 520, pseudo-likelihood = -13.85, time = 0.21s
[BernoulliRBM] Iteration 521, pseudo-likelihood = -13.70, time = 0.20s
[BernoulliRBM] Iteration 522, pseudo-likelihood = -13.77, time = 0.20s
[BernoulliRBM] Iteration 523, pseudo-likelihood = -13.95, time = 0.21s
[BernoulliRBM] Iteration 524, pseudo-likelihood = -13.87, time = 0.23s
[BernoulliRBM] Iteration 525, pseudo-likelihood = -13.80, time = 0.22s
[Berno

[BernoulliRBM] Iteration 628, pseudo-likelihood = -13.85, time = 0.21s
[BernoulliRBM] Iteration 629, pseudo-likelihood = -13.89, time = 0.21s
[BernoulliRBM] Iteration 630, pseudo-likelihood = -13.83, time = 0.21s
[BernoulliRBM] Iteration 631, pseudo-likelihood = -13.75, time = 0.20s
[BernoulliRBM] Iteration 632, pseudo-likelihood = -13.72, time = 0.20s
[BernoulliRBM] Iteration 633, pseudo-likelihood = -13.72, time = 0.21s
[BernoulliRBM] Iteration 634, pseudo-likelihood = -13.82, time = 0.22s
[BernoulliRBM] Iteration 635, pseudo-likelihood = -13.91, time = 0.21s
[BernoulliRBM] Iteration 636, pseudo-likelihood = -13.92, time = 0.21s
[BernoulliRBM] Iteration 637, pseudo-likelihood = -13.67, time = 0.21s
[BernoulliRBM] Iteration 638, pseudo-likelihood = -13.60, time = 0.21s
[BernoulliRBM] Iteration 639, pseudo-likelihood = -13.79, time = 0.21s
[BernoulliRBM] Iteration 640, pseudo-likelihood = -13.99, time = 0.21s
[BernoulliRBM] Iteration 641, pseudo-likelihood = -13.92, time = 0.21s
[Berno

[BernoulliRBM] Iteration 744, pseudo-likelihood = -13.86, time = 0.21s
[BernoulliRBM] Iteration 745, pseudo-likelihood = -13.74, time = 0.21s
[BernoulliRBM] Iteration 746, pseudo-likelihood = -13.69, time = 0.21s
[BernoulliRBM] Iteration 747, pseudo-likelihood = -13.74, time = 0.22s
[BernoulliRBM] Iteration 748, pseudo-likelihood = -13.70, time = 0.23s
[BernoulliRBM] Iteration 749, pseudo-likelihood = -13.76, time = 0.22s
[BernoulliRBM] Iteration 750, pseudo-likelihood = -13.85, time = 0.21s
[BernoulliRBM] Iteration 751, pseudo-likelihood = -13.75, time = 0.22s
[BernoulliRBM] Iteration 752, pseudo-likelihood = -13.64, time = 0.21s
[BernoulliRBM] Iteration 753, pseudo-likelihood = -13.73, time = 0.21s
[BernoulliRBM] Iteration 754, pseudo-likelihood = -13.78, time = 0.21s
[BernoulliRBM] Iteration 755, pseudo-likelihood = -13.83, time = 0.21s
[BernoulliRBM] Iteration 756, pseudo-likelihood = -13.80, time = 0.22s
[BernoulliRBM] Iteration 757, pseudo-likelihood = -13.81, time = 0.21s
[Berno

[BernoulliRBM] Iteration 860, pseudo-likelihood = -13.55, time = 0.24s
[BernoulliRBM] Iteration 861, pseudo-likelihood = -13.69, time = 0.27s
[BernoulliRBM] Iteration 862, pseudo-likelihood = -13.75, time = 0.23s
[BernoulliRBM] Iteration 863, pseudo-likelihood = -13.68, time = 0.21s
[BernoulliRBM] Iteration 864, pseudo-likelihood = -13.70, time = 0.22s
[BernoulliRBM] Iteration 865, pseudo-likelihood = -13.79, time = 0.22s
[BernoulliRBM] Iteration 866, pseudo-likelihood = -13.67, time = 0.24s
[BernoulliRBM] Iteration 867, pseudo-likelihood = -13.63, time = 0.21s
[BernoulliRBM] Iteration 868, pseudo-likelihood = -13.72, time = 0.21s
[BernoulliRBM] Iteration 869, pseudo-likelihood = -13.76, time = 0.21s
[BernoulliRBM] Iteration 870, pseudo-likelihood = -13.81, time = 0.22s
[BernoulliRBM] Iteration 871, pseudo-likelihood = -13.90, time = 0.20s
[BernoulliRBM] Iteration 872, pseudo-likelihood = -13.84, time = 0.20s
[BernoulliRBM] Iteration 873, pseudo-likelihood = -13.82, time = 0.20s
[Berno

[BernoulliRBM] Iteration 976, pseudo-likelihood = -13.73, time = 0.21s
[BernoulliRBM] Iteration 977, pseudo-likelihood = -13.78, time = 0.21s
[BernoulliRBM] Iteration 978, pseudo-likelihood = -13.75, time = 0.21s
[BernoulliRBM] Iteration 979, pseudo-likelihood = -13.72, time = 0.21s
[BernoulliRBM] Iteration 980, pseudo-likelihood = -13.74, time = 0.21s
[BernoulliRBM] Iteration 981, pseudo-likelihood = -13.85, time = 0.20s
[BernoulliRBM] Iteration 982, pseudo-likelihood = -13.89, time = 0.20s
[BernoulliRBM] Iteration 983, pseudo-likelihood = -13.88, time = 0.21s
[BernoulliRBM] Iteration 984, pseudo-likelihood = -13.75, time = 0.21s
[BernoulliRBM] Iteration 985, pseudo-likelihood = -13.65, time = 0.22s
[BernoulliRBM] Iteration 986, pseudo-likelihood = -13.66, time = 0.21s
[BernoulliRBM] Iteration 987, pseudo-likelihood = -13.67, time = 0.21s
[BernoulliRBM] Iteration 988, pseudo-likelihood = -13.69, time = 0.20s
[BernoulliRBM] Iteration 989, pseudo-likelihood = -13.71, time = 0.21s
[Berno

In [15]:
l = LogisticRegression(solver='newton-cg', multi_class='multinomial')
l.fit(X_train,y_train);
print("score on training: ",l.score(X_train,y_train))
print("score on test: ",l.score(X_test,y_test))

score on training:  0.483170466884
score on test:  0.519480519481
